In [1]:
import pandas as pd

In [2]:
file_path = "G:/academic/sem 7/Research - Multi-omic data in the cervical cancer studies/notebooks/HiSeqv/Before_undersample_HiSeqv.csv" 
df = pd.read_csv(file_path)

In [3]:
df

,SampleID,histological_type,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
0,TCGA-EA-A3HT-01,1,0.725318,0.189987,0.174594,0.496180,0.617883,0.637369,0.565425,0.609614,...,0.296442,0.000000,0.560584,0.255231,0.497724,0.626944,0.515713,0.222879,0.316847,0.405340
1,TCGA-FU-A3HZ-01,1,0.444151,0.248211,0.000000,0.561269,0.463589,0.230653,0.652436,0.324709,...,0.000000,0.290987,0.506421,0.479059,0.400047,0.302075,0.322926,0.267669,0.313092,0.530207
2,TCGA-FU-A3TQ-01,1,0.837766,0.230985,0.374522,0.554383,0.829937,0.389478,0.630412,0.516601,...,0.000000,0.000000,0.582022,0.705226,0.578096,0.420560,0.428795,0.444500,0.612588,0.361471
3,TCGA-DS-A0VK-01,1,0.564916,0.389652,0.450002,0.518949,0.531938,0.716303,0.719264,0.379869,...,0.000000,0.000000,0.446650,0.000000,0.369243,0.522974,0.751170,0.513679,0.660377,0.407865
4,TCGA-UC-A7PG-01,1,0.772081,0.413758,0.088769,0.665259,0.608024,0.331768,0.661789,0.486976,...,0.000000,0.115226,0.426500,0.225709,0.499123,0.588495,0.450644,0.511162,0.432341,0.405667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,TCGA-MY-A5BF-01,1,0.556710,0.122052,0.147576,0.556544,0.713526,0.665668,0.259738,0.277640,...,0.000000,0.000000,0.540330,0.000000,0.538678,0.523095,0.698141,0.184597,0.233885,0.423132
297,TCGA-EA-A556-01,0,0.388797,0.809501,0.000000,0.687726,0.501289,0.308968,0.821602,0.682432,...,0.000000,0.000000,0.280758,0.661005,0.545202,0.493125,0.562874,0.430963,0.260699,0.485921
298,TCGA-IR-A3LA-01,0,0.576657,0.273252,0.000000,0.478287,0.494694,0.659674,0.769528,0.449023,...,0.117572,0.089884,0.340676,0.224760,0.635093,0.261346,0.323549,0.764273,0.388770,0.801592
299,TCGA-VS-AA62-01,1,0.364161,0.200194,0.092233,0.569914,0.392592,0.609657,0.604375,0.396689,...,0.156602,0.000000,0.703145,0.475191,0.476143,0.684026,0.480346,0.681699,0.480234,0.454319


In [4]:
column_values = df['histological_type']
total_count = column_values.count()
value_counts = column_values.value_counts()
value_counts

histological_type
1    253
0     48
Name: count, dtype: int64

In [5]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier  # Import XGBoost
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming df is your DataFrame

xgb_classifier = XGBClassifier(objective='binary:logistic', random_state=42)  # Use binary:logistic for binary classification
K = np.arange(start=5, stop=90, step=5)
iterations = 10  # Number of iterations
feature_numbers = []
mean_accuracies = []

# Loop through the selected features
for idx, k in enumerate(K):
    accuracies_iter = []

    for iteration in range(iterations):
        # Feature selection using RFE
        X = df.iloc[:, 2:20132]
        y = df["histological_type"]

        selector = RFE(xgb_classifier, n_features_to_select=k)
        X_selected = selector.fit_transform(X, y)
        selected_features = X.columns[selector.support_]

        df_selected = df[['SampleID', 'histological_type'] + list(selected_features)].copy()

        # Undersample the dataset with a random seed between 0 and 100 for each iteration
        undersample_seed = np.random.randint(101)  # Generate a random seed between 0 and 100
        cervical_squamous_df = df_selected[df_selected['histological_type'] == 0]
        if len(cervical_squamous_df) > 48:
            cervical_squamous_df = resample(cervical_squamous_df, n_samples=48, random_state=undersample_seed)
        adeno_df = df_selected[df_selected['histological_type'] == 1]
        df2 = pd.concat([cervical_squamous_df, adeno_df])

        # Split the data into features (X) and target (y)
        X_selected = df2.iloc[:, 2:]
        y_selected = df2["histological_type"]

        # Perform 5-fold cross-validation
        scores = cross_val_score(xgb_classifier, X_selected, y_selected, cv=5, scoring='accuracy')

        # Calculate average accuracy across folds
        accuracy = np.mean(scores)
        accuracies_iter.append(accuracy)

        # Print update after each iteration
        print(f"Iteration {iteration + 1}/{iterations} for k={k}: Accuracy = {accuracy:.4f}, Undersample Seed = {undersample_seed}")

    # Store the feature number and its mean accuracy across iterations
    feature_numbers.append(k)
    mean_accuracies.append(np.mean(accuracies_iter))

    # Print update after each feature number
    print(f"Finished iteration for k={k}: Mean Accuracy = {mean_accuracies[-1]:.4f}\n")

# Plotting the feature numbers and their corresponding mean accuracies
plt.figure(figsize=(10, 6))
plt.plot(feature_numbers, mean_accuracies, marker='o', linestyle='-')

# Annotate each point with its accuracy
for i in range(len(feature_numbers)):
    plt.text(feature_numbers[i], mean_accuracies[i], f'{mean_accuracies[i]:.4f}', fontsize=8, ha='right', va='bottom')

plt.title('Mean Accuracy for each Feature Number (RFE + Undersampling + XGBoost with 5-Fold Cross-Validation)')
plt.xlabel('Number of Features')
plt.ylabel('Mean Accuracy')
plt.grid(True)
plt.show()



KeyboardInterrupt

